In [1]:
from model import *
from train_test_split import *
from predictionsGraph import *
from data_set import *
from torch.utils.data import DataLoader

In [2]:
seq_len = 3 #ovdje treba biti ista vrijednost kao kad se treniralo
batch_size = 1 #
hidden_dim = 1028
attention = True
train_data, val_data, test_data = load_data('real_for_all_podaci.csv') 
input_dim = train_data.shape[1]
train_dataset = Dataset(train_data, seq_len)
test_dataset = Dataset(test_data, seq_len)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=batch_size)
model = Net(input_dim,  hidden_dim = hidden_dim, seq_len=seq_len, attention)

In [3]:
path_to_model = 'models/epoch_41-batch_size_128-lr_0.0001-hidd_dim_1028.weights'
model.load_state_dict(torch.load(path_to_model))
model.eval()

Net(
  (lstm): LSTM(7, 1028, batch_first=True)
  (fc1): Linear(in_features=1028, out_features=1028, bias=True)
  (fc): Linear(in_features=1028, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (attention): Attention(
    (v): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [4]:
realValues = []
predictedValues = []
#h = model.lstm.weight_ih_l0
#h = model.init_hidden(batch_size)

for (inputs, labels) in test_loader:
    inputs, labels = inputs, labels 
    print(inputs.shape)
    output = model(inputs)#, h)
    realValues.append(labels.item())
    predictedValues.append(output.item())
    
predictionsGraph(realValues, predictedValues)
predictionsGraphScatter(realValues, predictedValues)

torch.Size([1, 3, 7])


TypeError: lstm() received an invalid combination of arguments - got (Tensor, tuple, list, bool, bool, float, bool, bool, bool), but expected one of:
 * (Tensor data, Tensor batch_sizes, tuple of Tensors hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional)
      didn't match because some of the arguments have invalid types: (Tensor, !tuple!, !list!, !bool!, bool, !float!, !bool!, bool, bool)
 * (Tensor input, tuple of Tensors hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional, bool batch_first)
      didn't match because some of the arguments have invalid types: (Tensor, !tuple!, !list!, bool, !bool!, float, bool, bool, bool)
